In [1]:
from embeddings import get_token_embeddings_batch

In [2]:
from model import load_model

tokenizer, model = load_model()

/home/mihirathale/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [76]:
import json

docs = json.load(open("context_paras.json"))

In [72]:
docs = list(docs.values())

In [75]:
import random

doc_ids = random.sample([i for i in range(len(docs))], 1000)

In [77]:
titles = list(docs.keys())
docs = list(docs.values())

In [78]:
documents = []
titles_ = []

for i in doc_ids:
    documents.append(docs[i])
    titles_.append(titles[i])

In [80]:
len(documents), len(titles_)

(1000, 1000)

In [82]:
documents[0], titles_[0]

('Rockstar Games Social Club (Social Club) is a digital rights management, multiplayer and communications service provided by Rockstar Games for use with their latest generation of games.  Rockstar Games Social Club was first announced on 27 March 2008, with pre-registration beginning on 14 April 2008.  However, that date was moved to 17 April 2008.  The name is a reference to organized crime, which commonly uses the term "social club" to describe a meeting place or hideout.  The service received a major update in 2012, prior to the release of "Max Payne 3", with the addition of social networking features and a "crews" system that allows players to form groups and combine their achievements to unlock bonus features.',
 'Rockstar Games Social Club')

In [83]:
with open("documents.json", "w") as f:
    json.dump(documents, f)

with open("titles.json", "w") as f:
    json.dump(titles_, f)


In [84]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from transformers import AutoTokenizer, AutoModel

In [88]:
loader = PyPDFLoader("Harry.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2"),
    chunk_size=256,
    chunk_overlap=16,
    strip_whitespace=True,
)
docs = text_splitter.split_documents(pages)


In [53]:
docs[0]

Document(page_content='When a letter arrives for unhappy but  \nordinary Harry Potter, a decade-old secret  \nis revealed to him. His parents were  \nwizards, killed by a Dark Lord’s curse  \nwhen Harry was just a baby, and which he  \nsomehow survived. Escaping from his  \nunbearable Muggle guardians to Hogwarts,  \na wizarding school brimming with ghosts  \nand enchantments, Harry stumbles into a  \nsinister adventure when he finds a three- \nheaded dog guarding a room on the third  \nfloor. Then he hears of a missing stone  \nwith astonishing powers which could be  \nvaluable, dangerous, or both.  \n \n‘Funny, imaginative, magical ... Rowling has  \nwoken up a whole generation to reading. In  \nthe 2020s, thirty-something book-lovers  \nwill know each other by smug references  \nto Diagon Alley and Quidditch’ The Times', metadata={'source': 'Harry.pdf', 'page': 1})

In [89]:
doc_content = [doc.page_content for doc in docs]

In [90]:
gryffindor_paragraphs = [
    "Gryffindor House values courage, bravery, and determination above all. Its emblematic animal is the lion, symbolizing fierce loyalty and boldness.",
    "The common room of Gryffindor is located in a cozy tower, guarded by the Fat Lady who requires a password to enter. Inside, warm fireplaces and scarlet tapestries welcome the daring students.",
    "Harry Potter, Hermione Granger, and Ron Weasley all belonged to Gryffindor, making the house famous among Hogwarts alumni. Their heroic feats embody the house’s core values.",
    "The house founder, Godric Gryffindor, was known for pulling his sword from the Sorting Hat in times of need. His legacy of bravery still empowers young witches and wizards.",
    "Gryffindors are often seen as reckless by other houses, but they see it as bold action in the face of fear. They’re known to charge into danger when others hesitate.",
    "The Gryffindor table in the Great Hall is always filled with laughter, loud cheers, and friendly competition. House pride runs deep, especially during the annual Quidditch Cup.",
    "Gryffindor’s rivalry with Slytherin fuels much of Hogwarts’ drama, dating back to disagreements between Godric and Salazar. These tensions still echo in friendly (and sometimes not-so-friendly) duels.",
    "Scarlet and gold, the house colors, are worn with pride during every celebration and sporting event. The lion roars not just from the banners but from every student’s heart.",
    "Bravery in Gryffindor doesn't always mean fighting dark wizards; it can be standing up to your friends or speaking truth to power. Even quiet courage is honored within the tower.",
    "First years sorted into Gryffindor often feel an immediate sense of belonging, inspired by the tales of past heroes. From the moment they wear the lion's badge, they begin their legacy."
]


In [91]:
doc_content.extend(gryffindor_paragraphs)


In [92]:
with open("harry_docs.json", "w") as f:
    json.dump(doc_content, f)

In [43]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

query_embeddings = embedding_model.embed_query(doc_content[0])

In [93]:
doc_embeddings = embedding_model.embed_documents(doc_content)

In [50]:
import numpy as np

element_wise = np.array(query_embeddings) * np.array(doc_embeddings[0])

In [94]:
len(doc_embeddings)

707

In [52]:
element_wise.shape

(3072,)

In [97]:
import numpy as np

numpy_embeddings = np.array(doc_embeddings)

In [98]:
np.save("embeddings.npy", numpy_embeddings)